In [ ]:
# LLM model call (in LangChain)
# Chat model vs LLM (both are in LangChain)

#from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

#llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI()

#a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

# ChatOpenAI is cheaper and more chatable than OpenAI. so don't use OpenAI anymore. (+ also OpenAI is due-dated) 
b

'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.2
)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}. Also, what is your name?"
    )
prompt = template.format(country_a="Mexico", country_b="Thailand")


chat.predict(prompt)




'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles). My name is Assistant.'

In [5]:
# from langchain.schema import HumanMessage, AIMessage, SystemMessage (if you use ChatPromptTemplate, you don't need this library)
# Prompt Templates : use placeholder{} and exchage country, name, etc.. from message list
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply only {lang}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])
# SystemMessage is to setting for LLM

prompt = template.format_messages(
    lang="Korean",
    name="Kim Soo Hyun",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='멕시코와 태국 사이의 거리는 대략 15,000km입니다. 저는 김수현입니다.')

In [6]:
# Output parser : We need this because sometimes you have to transform 'LLM Response' 
# we gonna make response 'text(string)' type to 'list'
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    # Outputparser should implment def 'parse'!
    def parse(self, text):
        items =  text.strip().split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()
p.parse("Hello,how,    are,you")



['Hello', 'how', 'are', 'you']

In [7]:
# We already import 'AIMessagefrom langchain.prompts ..' so pass.
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowerecase. DO NOT reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items = 10,
    question = "What are the colors?"
)

result = chat.predict_messages(prompt)

# Using Outputparser
p.parse(result.content)

# Ok. cool. But this Code is toooo long and lot a things to do. 
# Let's change this code with 'LangChain Expression Language'!


['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'black',
 'white',
 'pink',
 'brown']

In [8]:
# LangChain Expression Language
# template, prompt and predict. These thigs can exchange this. only one code can replace these three things.
# We only need 'ChatOpenAI (already done) / CommaOutputParser (already done) / ChatPromptTemplate (doing now)
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowerecase. DO NOT reply with anything else."),
    ("human", "{question}")
])

chain_one = template | chat | CommaOutputParser() #create 'chain' for LangChain
#chain_two = template2 | chat | outputparser
#all  = chain_one | chain_two | output..

chain_one.invoke({
    "max_items": 5,
    "question": "Wat are the pokemons?"
})

# LangChain library is amazing

['pikachu', 'charmander', 'bulbasaur', 'squirtle']

In [9]:
# Using LangChain from zero(=nothing)

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, 
                  streaming=True, 
                  callbacks=[StreamingStdOutCallbackHandler()]) # you can see strings(log..?) what LLM doing now

# chain + chain expression
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. "
    "You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

vegan_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. "
    "You find alternative ingredients and explain their preparation. "
    "You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
    ("human", "{recipe}")
])

vegan_chain = vegan_chef_prompt | chat


final_chain = {"recipe": chef_chain} | vegan_chain

final_chain.invoke({
    "cuisine": "indian"
})


Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground turmeric
- 1 teaspoon garam masala
- 1 teaspoon ground coriander
- 1 teaspoon chili powder (adjust to taste)
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- Salt and pepper to taste
- 2 tablespoons oil
- 1 onion, finely chopped
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, turmeric, garam masala, coriander, chili powder, garlic, ginger, salt, and pepper. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or 

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare the tofu or paneer to use in this recipe:\n\nTofu:\n1. Use extra-firm tofu and press it to remove excess water. You can do this by wrapping the tofu block in a clean kitchen towel and placing a heavy object on top for about 15-30 minutes.\n2. Cut the tofu into bite-sized cubes and follow the marinade instructions in the recipe. Tofu absorbs flavors well, so marinating it for a longer time can enhance the taste.\n\nPaneer:\n1. If you prefer paneer, you can either make it at home or buy it from the store. Cut the paneer into cubes and follow the marinade instructions in the recipe.\n2. Paneer is delicate, so marinate it for a shorter time compared to tofu to prevent it from breaking apart.\n\nFollow the rest of the recipe as it is, substituting the chicken with the marinated tofu or paneer during the coo

In [10]:
# Model I/O : FewShotPromptTemplate (important!!)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

#t = PromptTemplate.from_template("What is the capital of {country}")
# this method is shortcut. Let's see longcut(explain shortcut)

t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"]
)

t.format(country="France")

'What is the capital of France'

In [11]:
# OK. Now, What is FewShot??
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# show example model what you want => format examples
examples = [
    {
    "question": "What do you know about France?",
    "answer": """
    Here is what I know:
    Capital: Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro
    """,
    },
    {
    "question": "What do you know about Italy?",
    "answer": """
    I know this:
    Capital: Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency: Euro
    """,
    },
    {
    "question": "What do you know about Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Food: Souvlaki and Feta Cheese
    Currency: Euro
    """,
    },
]

example_template = """
    Human:{question}
    AI:{answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human:What do you know about {country}?",
    input_variables=["country"]
)

chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

AI:
I know the following:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='AI:\nI know the following:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

In [12]:
# How to use Fewshot with chatbot
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# show example model what you want => format examples
examples = [
    {
    "country": "France?",
    "answer": """
    Here is what I know:
    Capital: Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro
    """,
    },
    {
    "country": "Italy?",
    "answer": """
    I know this:
    Capital: Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency: Euro
    """,
    },
    {
    "country": "Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Food: Souvlaki and Feta Cheese
    Currency: Euro
    """,
    },
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "what do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."
     "Answer in 6-8 short bullet points. include:"
     "Capital, language, currency, neighbors, climate, major landforms, and one notable geological feature."),
    example_prompt,
    ("human", "What do you know about {country}?")
])
chain = final_prompt | chat

chain.invoke({
    "country": "Korea"
})


    I know this:
    Capital: Seoul
    Language: Korean
    Currency: South Korean Won
    Neighbors: North Korea, China, Japan
    Climate: Temperate with four distinct seasons
    Major Landforms: Korean Peninsula, Taebaek Mountains, Nakdong River
    Notable Geological Feature: Jeju Island, a volcanic island off the southern coast

AIMessageChunk(content='\n    I know this:\n    Capital: Seoul\n    Language: Korean\n    Currency: South Korean Won\n    Neighbors: North Korea, China, Japan\n    Climate: Temperate with four distinct seasons\n    Major Landforms: Korean Peninsula, Taebaek Mountains, Nakdong River\n    Notable Geological Feature: Jeju Island, a volcanic island off the southern coast')

In [13]:
# Sometimes Soooo Many examples exists. So We need to choose what example use and what example don't use
# Let's make how to
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector



examples = [
    {
    "country": "France?",
    "answer": """
    Here is what I know:
    Capital: Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro
    """,
    },
    {
    "country": "Italy?",
    "answer": """
    I know this:
    Capital: Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency: Euro
    """,
    },
    {
    "country": "Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Food: Souvlaki and Feta Cheese
    Currency: Euro
    """,
    },
]



class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]
    

example_template = """
    Human:{country}
    AI:{answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

# example_selector = LengthBasedExampleSelector(
#     example_prompt=example_prompt,
#     examples=examples,
#     max_length=10
# )

example_selector = RandomExampleSelector(
    examples=examples
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human:What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Korea")



'\n    Human:France?\n    AI:\n    Here is what I know:\n    Capital: Paris\n    Language: French\n    Food: Wine and Cheese\n    Currency: Euro\n    \n\n\n\n    Human:Italy?\n    AI:\n    I know this:\n    Capital: Rome\n    Language: Italian\n    Food: Pizza and Pasta\n    Currency: Euro\n    \n\n\n\n    Human:Greece?\n    AI:\n    I know this:\n    Capital: Athens\n    Language: Greek\n    Food: Souvlaki and Feta Cheese\n    Currency: Euro\n    \n\n\nHuman:What do you know about Korea?'

In [14]:
# from langchain.prompts import load_prompt
# if you have many prompts and have to get together all at once, use below library
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
"""
You are a role playing assistant.
And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
"""
This is an example of how you talk:

Human: {example_question}
You: {example_answer}
"""
)

start = PromptTemplate.from_template(
"""
Start now!

Human: {question}
You:
"""
)

final = PromptTemplate.from_template(
"""
{intro}

{example}

{start}
"""
)


prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final,
                                     pipeline_prompts=prompts)

# prompt = load_prompt("./prompt.yaml")

full_prompt.format(character="Pirate",
              example_question="What is your location?",
              example_answer="Arrrg! That is a secret! Arg!!",
              question="What is your fav food?")

chain = full_prompt | chat

chain.invoke({
    "character":"Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrrg! That is a secret! Arg!!",
    "question":"What is your fav food?"
})

Arrrg! Me favorite grub be a hearty plate of salted fish and hardtack! Aye, it be a meal fit for a pirate like meself! Arrr!

AIMessageChunk(content='Arrrg! Me favorite grub be a hearty plate of salted fish and hardtack! Aye, it be a meal fit for a pirate like meself! Arrr!')

In [15]:
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1
)
set_debug(True)
chat.predict("How do you make italian pasta?")



[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or rolling pin.\n7. Cut the pasta sheets into desired shapes such as fettucc

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or rolling pin.\n7. Cut the pasta sheets into desired shapes such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [16]:
chat.predict("How do you make italian pasta?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or rolling pin.\n7. Cut the pasta sheets into desired shapes such as fettucc

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or rolling pin.\n7. Cut the pasta sheets into desired shapes such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [ ]:
# How much money spend when we call api?
from langchain.callbacks import get_openai_callback

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju?")
    b = chat.predict("What is the recipe for kimchi?")
    print(a,b,"\n")
    print(usage)


[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Ingredients:\n- 1 cup of rice\n- 1 cup of barley\n- 1 cup of water\n- 1 tablespoon of yeast\n- 1 tablespoon of sugar\n\nInstructions:\n1. Rinse the rice and barley thoroughly and soak them in water for at least 4 hours or overnight.\n2. Drain the rice and barley and steam them until they are fully cooked.\n3. In a large bowl, mix the cooked rice and barley with water, yeast, and sugar. Cover the bowl with a clean cloth and let it ferment for 1-2 weeks in a warm place.\n4. After the fermentation process is complete, strain the mixture through a cheesecloth to remove any solids.\n5. Transfer the liquid to a clean container and let it sit for another 1-2 weeks to allow the flavors to develop.\n6. Once the soju is ready, strain it again and transfer it to 